<h1> Quantitative Measures for Gesture Space </h1>

In this module, we learn different ways to measure the space used by gestures. This notebook is based on https://envisionbox.org/embedded_Analysis_kinematic_features_module.html

<h2> Overview of the script </h2>

1. Vertical Amplitude
2. McNeillian Space
3. Volumetric Space

In [1]:
import os
import numpy as np
import pandas as pd



In [2]:
df = pd.read_csv('p:/shared/FOCUS-GROUPS/Gesture-Kinematics/gesture_space_compuation/1130_JS_body.csv')

In [3]:
df.head()

,Unnamed: 0,time,X_NOSE,Y_NOSE,Z_NOSE,visibility_NOSE,X_LEFT_EYE_INNER,Y_LEFT_EYE_INNER,Z_LEFT_EYE_INNER,visibility_LEFT_EYE_INNER,...,Z_RIGHT_HEEL,visibility_RIGHT_HEEL,X_LEFT_FOOT_INDEX,Y_LEFT_FOOT_INDEX,Z_LEFT_FOOT_INDEX,visibility_LEFT_FOOT_INDEX,X_RIGHT_FOOT_INDEX,Y_RIGHT_FOOT_INDEX,Z_RIGHT_FOOT_INDEX,visibility_RIGHT_FOOT_INDEX
0,1,0.000000,0.494983,0.386290,-0.431344,0.999973,0.501042,0.362740,-0.423350,0.999938,...,-0.093605,0.624468,0.552150,0.834183,-0.066258,0.439369,0.459295,0.890626,-0.174257,0.757300
1,2,33.333333,0.495307,0.384873,-0.282090,0.999979,0.501509,0.360980,-0.277935,0.999978,...,-0.172937,0.751992,0.478470,0.825913,-0.175029,0.644546,0.528809,0.842235,-0.229534,0.834528
2,3,66.666667,0.496901,0.383783,-0.370449,0.999988,0.502612,0.359785,-0.363854,0.999985,...,-0.079223,0.807920,0.551735,0.854201,-0.138214,0.825628,0.500692,0.846941,-0.142319,0.893050
3,4,100.000000,0.494433,0.379609,-0.478178,0.999978,0.500472,0.355881,-0.467679,0.999916,...,-0.042131,0.812141,0.576784,0.871483,-0.027148,0.591543,0.414131,0.913681,-0.139862,0.883535
4,5,133.333333,0.493474,0.374980,-0.473152,0.999983,0.499924,0.352251,-0.462004,0.999949,...,-0.045055,0.740282,0.577629,0.868587,-0.106867,0.637964,0.435673,0.914415,-0.141587,0.824106


In [4]:

def convert_MP_to_OP(df_MP):
    # first we create a dictionary that maps the names in our MediaPipe output to the names in our OpenPose output
    conv_dict = {"RIGHT_WRIST":"R_Hand", "LEFT_WRIST":"L_Hand","NOSE":"Nose","RIGHT_ELBOW":"RElb","LEFT_ELBOW":"LElb","RIGHT_HIP":"RHip","LEFT_HIP":"LHip", "LEFT_EYE":"LEye","RIGHT_EYE":"REye"}
    
    OP_df = pd.DataFrame()
    
    for key in conv_dict:
        
        OP_df[conv_dict[key]] = [[row["X_"+key],row["Y_"+key],row["Z_"+key]] for _,row in df_MP.iterrows()]
    OP_df["time"] = df_MP["time"].copy()
    
    # NOTE: not all methods track the exact same keypoints. For some of our calculations we need a Neck point, and a Mid-Hip Point.
    # We need to calculate these based on others
    OP_df["Neck"] = [[np.mean([row["X_LEFT_SHOULDER"],row["Y_RIGHT_SHOULDER"]]),np.mean([row["X_LEFT_SHOULDER"],row["Y_LEFT_SHOULDER"]]),row["Z_RIGHT_SHOULDER"] ] for _, row in df_MP.iterrows()]
    OP_df["MidHip"] = [[np.mean([row["X_LEFT_HIP"],row["X_RIGHT_HIP"]]),np.mean([row["Y_LEFT_HIP"],row["Y_LEFT_HIP"]]),row["Z_RIGHT_HIP"] ] for _, row in df_MP.iterrows()]
    return OP_df

    

In [5]:
df_OP = convert_MP_to_OP(df)

In [6]:
df_OP.head()

,R_Hand,L_Hand,Nose,RElb,LElb,RHip,LHip,LEye,REye,time,Neck,MidHip
0,"[0.496747642755508, 0.663687169551849, -0.3303...","[0.518324732780457, 0.657763123512268, -0.3496...","[0.494983106851578, 0.386290222406387, -0.4313...","[0.447110593318939, 0.576705694198608, -0.2559...","[0.553987324237823, 0.57297694683075, -0.26294...","[0.473699182271957, 0.625439465045929, 0.00122...","[0.527776718139648, 0.619546949863434, -0.0012...","[0.505005955696106, 0.361148864030838, -0.4233...","[0.485006630420685, 0.363706916570663, -0.4215...",0.000000,"[0.491848096251488, 0.48940247297287, -0.24001...","[0.5007379502058025, 0.619546949863434, 0.0012..."
1,"[0.490709483623505, 0.663850605487823, -0.1924...","[0.515055358409882, 0.660858452320099, -0.2108...","[0.495306670665741, 0.38487297296524, -0.28209...","[0.446966260671616, 0.577283918857574, -0.1193...","[0.554111182689667, 0.575322270393372, -0.1445...","[0.472991049289703, 0.632175087928772, 0.00907...","[0.528351783752441, 0.627220332622528, -0.0091...","[0.505473017692566, 0.359289705753326, -0.2779...","[0.485719680786133, 0.36245608329773, -0.27502...",33.333333,"[0.49196985363960255, 0.4899606108665465, -0.1...","[0.5006714165210719, 0.627220332622528, 0.0090..."
2,"[0.486942857503891, 0.66176849603653, -0.26786...","[0.515659093856812, 0.655936360359192, -0.2989...","[0.496900975704193, 0.383783131837845, -0.3704...","[0.447020471096039, 0.575847744941711, -0.1959...","[0.555079102516174, 0.574874043464661, -0.2190...","[0.473819434642792, 0.626804292201996, 0.00607...","[0.528030335903168, 0.619612812995911, -0.0061...","[0.506270945072174, 0.358135730028152, -0.3638...","[0.486639589071274, 0.3616524040699, -0.362132...",66.666667,"[0.491930112242699, 0.489900901913643, -0.1933...","[0.50092488527298, 0.619612812995911, 0.006075..."
3,"[0.487284362316132, 0.658783316612244, -0.3624...","[0.517875254154205, 0.657841682434082, -0.3992...","[0.494432508945465, 0.379608631134033, -0.4781...","[0.446794509887695, 0.571514666080475, -0.2826...","[0.554669260978699, 0.572065114974976, -0.3021...","[0.472307741641998, 0.628111898899078, 0.00282...","[0.526997268199921, 0.623744964599609, -0.0028...","[0.504416525363922, 0.355045169591904, -0.4676...","[0.485781282186508, 0.358441472053528, -0.4665...",100.000000,"[0.49072588980197906, 0.4891711473464965, -0.2...","[0.4996525049209595, 0.623744964599609, 0.0028..."
4,"[0.485915035009384, 0.659861207008362, -0.3548...","[0.518423736095428, 0.658816158771515, -0.3982...","[0.493473947048187, 0.374979823827744, -0.4731...","[0.447036981582642, 0.572198033332825, -0.2720...","[0.554447710514069, 0.572217464447021, -0.2961...","[0.471505105495453, 0.628473997116089, 0.00243...","[0.525667369365692, 0.62555867433548, -0.00248...","[0.504178583621979, 0.351745873689651, -0.4620...","[0.484687656164169, 0.354983419179916, -0.4600...",133.333333,"[0.48998552560806297, 0.489640295505524, -0.26...","[0.4985862374305725, 0.62555867433548, 0.00243..."


<h2> Vertical Amplitude </h2>


This first feature, vertical amplitude, will calculate the maximum amplitude of the hands. It does this not in an image-specific way (such as pixels, meters), but in a person-specific way, giving you the max height relative to the body of the person performing the gesture.

In [7]:

def calc_vert_height(df, hand):
    # Vertical amplitude
    # H: 0 = below midline;
    #    1 = between midline and middle-upper body;
    #    2 = above middle-upper body, but below shoulders;
    #    3 = between shoulders nad middle of face;
    #    4 = between middle of face and top of head;
    #    5 = above head

    H = []
    for index, frame in df.iterrows():
        SP_mid = ((df.loc[index, "Neck"][1] - df.loc[index, "MidHip"][1]) / 2) + df.loc[index, "MidHip"][1]
        Mid_up = ((df.loc[index, "Nose"][1] - df.loc[index, "Neck"][1]) / 2) + df.loc[index, "Neck"][1]
        Eye_mid = (df.loc[index, "REye"][1] + df.loc[index, "LEye"][1] / 2)  # mean of the two eyes vert height
        Head_TP = ((df.loc[index, "Nose"][1] - Eye_mid) * 2) + df.loc[index, "Nose"][1]

        if hand == "B":
            hand_height = max([df.loc[index, "R_Hand"][1], df.loc[index, "L_Hand"][1]])
        else:
            hand_str = hand + "_Hand"
            hand_height = df.loc[index][hand_str][1]

        if hand_height > SP_mid:
            if hand_height > Mid_up:
                if hand_height > df.loc[index, "Neck"][1]:
                    if hand_height > df.loc[index, "Nose"][1] :
                        if hand_height > Head_TP:
                            H.append(5)
                        else:
                            H.append(4)
                    else:
                        H.append(3)
                else:
                    H.append(2)
            else:
                H.append(1)
        else:
            H.append(0)
    MaxHeight = max(H)
    return MaxHeight




In [8]:
max_height_R = calc_vert_height(df_OP, "B")

print("Max height for the two hands: " + str(max_height_R))

Max height for the two hands: 5


We get a value of 1, indicating that the maximum height (considering both hands) is "between midline and middle-upper body". The function gives the option of indicating whether you want to consider one hand, or both, which can be useful if you've annotated the handedness of a gesture. In this case, we know that this is a two-handed gesture, so we'll use the "B" tag for our calculations.

<h2>McNeillian Space</h2>


What if we want to be more specific? Those familiar with gesture studies will likely have seen David McNeill's (1992) delineation of <i>gesture space</i>. 
<br><img src="./gesture_space.jpg"></center><br>
The gesture space offers some interesting insights into the way we use visual space during a gesture. For example, is it produces primarily directly in front of us? Do we cover a lot of space around us? How 'expansive' is the gesture? However, these can be difficult or time consuming to manually annotate, and even more so if the video angle is not straight ahead. However, we can implement this using our motion tracking keypoints.<br>
This requires several calculations, such as defining the grid seen in the image above, checking the position of the hands in relation to the grid, and calculating different features about where the hands moved, which areas they occupied, etc. 

In [9]:
import statistics

def calc_mcneillian_space(df, hand_idx):
    # this calls the define_mcneillian_grid function for each frame, then assign the hand to one space for each frame
    # output:
    # space_use - how many unique spaces were traversed
    # mcneillian_max - outer-most main space entered
    # mcneillian_mode - which main space was primarily used
    # 1 = Center-center
    # 2 = Center
    # 3 = Periphery
    # 4 = Extra-Periphery
    # subsections for periphery and extra periphery:
    # 1 = upper right
    # 2 = right
    # 3 = lower right
    # 4 = lower
    # 5 = lower left
    # 6 = left
    # 7 = upper left
    # 8 = upper
    if hand_idx == 'B':
        hands = ['L_Hand','R_Hand']
    else:
        hands = [hand_idx + '_Hand']
    # compare, at each frame, each hand to the (sub)section limits, going from inner to outer, clockwise
    for hand in hands:
        Space = []

        for frame in range(len(df)):

            cc_xmin, cc_xmax, cc_ymin, cc_ymax, c_xmin, c_xmax, c_ymin, c_ymax, p_xmin, p_xmax, p_ymin, p_ymax = \
            define_mcneillian_grid(df, frame)
            # centre-centre
            if cc_xmin < df[hand][frame][0] < cc_xmax and cc_ymin < df[hand][frame][1] < cc_ymax:
                Space.append(1)
            # centre
            elif c_xmin < df[hand][frame][0] < c_xmax and c_ymin < df[hand][frame][1] < c_ymax:
                Space.append(2)
            # periph
            elif p_xmin < df[hand][frame][0] < p_xmax and p_ymin < df[hand][frame][1] < p_ymax:
                # if it's in the periphery, we need to also get the subsection
                # first, is it on the right side?
                if cc_xmax < df[hand][frame][0]:
                    # if so, we narrow down the y location
                    if cc_ymax < df[hand][frame][1]:
                        Space.append(31)
                    elif cc_ymin < df[hand][frame][1]:
                        Space.append(32)
                    else:
                        Space.append(33)
                elif cc_xmin < df[hand][frame][0]:
                    if c_ymax < df[hand][frame][1]:
                        Space.append(38)
                    else:
                        Space.append(34)
                else:
                    if cc_ymax < df[hand][frame][1]:
                        Space.append(37)
                    elif cc_ymin < df[hand][frame][1]:
                        Space.append(36)
                    else:
                        Space.append(35)
            else:  # if it's not periphery, it has to be extra periphery. We just need to get subsections
                if c_xmax < df[hand][frame][0]:
                    if cc_ymax < df[hand][frame][1]:
                        Space.append(41)
                    elif cc_ymin < df[hand][frame][1]:
                        Space.append(42)
                    else:
                        Space.append(43)
                elif cc_xmin < df[hand][frame][0]:
                    if c_ymax < df[hand][frame][1]:
                        Space.append(48)
                    else:
                        Space.append(44)
                else:
                    if c_ymax < df[hand][frame][1]:
                        Space.append(47)
                    elif c_ymin < df[hand][frame][1]:
                        Space.append(46)
                    else:
                        Space.append(45)
        if hand == 'L_Hand':
            Space_L = Space
        else:
            Space_R = Space

    # how many spaces used?
    if hand_idx == 'L' or hand_idx == 'B':
        space_use_L = len(set(Space_L))
        if max(Space_L) > 40:
            mcneillian_maxL = 4
        elif max(Space_L) > 30:
            mcneillian_maxL = 3
        else:
            mcneillian_maxL = max(Space_L)
        # which main space was most used?
        mcneillian_modeL = get_mcneillian_mode(Space_L)
    else:
        space_use_L = 'NA'
        mcneillian_maxL = 'NA'
        mcneillian_modeL = 'NA'

    if hand_idx == 'R' or hand_idx == 'B':
        space_use_R = len(set(Space_R))
        # maximum distance (main spaces)
        if max(Space_R) > 40:
            mcneillian_maxR = 4
        elif max(Space_R) > 30:
            mcneillian_maxR = 3
        else:
            mcneillian_maxR = max(Space_R)
        # which main space was most used?
        mcneillian_modeR = get_mcneillian_mode(Space_R)
    else:
        space_use_R = 'NA'
        mcneillian_maxR = 'NA'
        mcneillian_modeR = 'NA'

    return space_use_L, space_use_R, mcneillian_maxL, mcneillian_maxR, mcneillian_modeL, mcneillian_modeR


def get_mcneillian_mode(spaces):
    mainspace = []
    for space in spaces:
        if space > 40:
            mainspace.append(4)
        elif space > 30:
            mainspace.append(3)
        else:
            mainspace.append(space)

    mcneillian_mode = statistics.mode(mainspace)
    return mcneillian_mode

def define_mcneillian_grid(df, frame):
    # define the grid based on a single frame, output xmin,xmax, ymin, ymax for each main section
    # subsections can all be found based on these boundaries
    bodycent = df['Neck'][frame][1] - (df['Neck'][frame][1] - df['MidHip'][frame][1])/2
    face_width = (df['LEye'][frame][0] - df['REye'][frame][0])*2
    body_width = df['LHip'][frame][0] - df['RHip'][frame][0]

    # define boundaries for center-center
    cc_xmin = df['RHip'][frame][0]
    cc_xmax = df['LHip'][frame][0]
    cc_len = cc_xmax - cc_xmin
    cc_ymin = bodycent - cc_len/2
    cc_ymax = bodycent + cc_len/2

    # define boundaries for center
    c_xmin = df['RHip'][frame][0] - body_width/2
    c_xmax = df['LHip'][frame][0] + body_width/2
    c_len = c_xmax - c_xmin
    c_ymin = bodycent - c_len/2
    c_ymax = bodycent + c_len/2

    # define boundaries of periphery
    p_ymax = df['LEye'][frame][1] + (df['LEye'][frame][1] - df['Nose'][frame][1])
    p_ymin = bodycent - (p_ymax - bodycent) # make the box symmetrical around the body center
    p_xmin = c_xmin - face_width
    p_xmax = c_xmax + face_width

    return  cc_xmin, cc_xmax, cc_ymin, cc_ymax, c_xmin, c_xmax, c_ymin, c_ymax, p_xmin, p_xmax, p_ymin, p_ymax



In [10]:
space_use_L, space_use_R, mcneillian_maxL, mcneillian_maxR, mcneillian_modeL, mcneillian_modeR = calc_mcneillian_space(df_OP, "B")

print("Number of spaces uses by the right hand: " + str(space_use_R))
print("Most peripheral space used by right hand: " + str(mcneillian_maxR))
print("Right hand spent most time in space number: " + str(mcneillian_modeR))

Number of spaces uses by the right hand: 7
Most peripheral space used by right hand: 4
Right hand spent most time in space number: 4


As we see, 4 different spaces were used (centre, 2 in periphery, and 1 in extra-periphery, in this case). Additionally, space 4 (extra periphery) is both the maximally peripheral space used, and where the right hand spent mos of its time.<br>


<h2>Volumetric Space</h2>

Another way to think about the space used is to calculate the volumetric space. Imagine that at the beginning of your annotation, we draw a cube (if the data is 3D, otherwise a box) around the hands, such that the hands position along the x-axis forms the outer side-limits of the cube/box, and their position on the y-axis forms the upper and lower limits. Our cube/box will therefore be quite flat at the beginning. But if we update and expand this cube/box with each frame, we can get an idea of the dynamic space that is used during a gesture.

In [11]:

def calc_volume_size(df, hand):
    # calculates the volumetric size of the gesture, ie how much visual space was utlized by the hands
    # for 3D data, this is actual volume (ie. using z-axis), for 2D this is area, using only x and y\
    # first we check if we should use one or both hands for calculating the initial boundaries
    if hand == 'B':
        x_max = max([df['R_Hand'][0][0], df['L_Hand'][0][0]])
        x_min = min([df['R_Hand'][0][0], df['L_Hand'][0][0]])
        y_max = max([df['R_Hand'][0][0], df['L_Hand'][0][1]])
        y_min = min([df['R_Hand'][0][0], df['L_Hand'][0][1]])
    else:
        hand_str = hand + '_Hand'
        x_min = df[hand_str][0][0]
        x_max = df[hand_str][0][0]
        y_min = df[hand_str][0][1]
        y_max = df[hand_str][0][1]
    # then we check if it's 3D or 2D data
    if len(df['R_Hand'][0]) > 2:
        if hand == 'B':
            z_max = max([df['R_Hand'][0][2], df['L_Hand'][0][2]])
            z_min = min([df['R_Hand'][0][2], df['L_Hand'][0][2]])
        else:
            z_min = df[hand_str][0][2]
            z_max = df[hand_str][0][2]
    # at each frame, compare the current min and max with the previous, to ultimately find the outer values
    if hand == 'B':
        hand_list = ['R_Hand', 'L_Hand']
    else:
        hand_list = [hand_str]

    for frame in range(1, len(df)):
        for hand_idx in hand_list:
            if df[hand_idx][frame][0] < x_min:
                x_min = df[hand_idx][frame][0]
            if df[hand_idx][frame][0] > x_max:
                x_max = df[hand_idx][frame][0]
            if df[hand_idx][frame][0] < y_min:
                y_min = df[hand_idx][frame][1]
            if df[hand_idx][frame][0] > y_max:
                y_max = df[hand_idx][frame][1]
            if len(df[hand_idx][0]) > 2:
                if df[hand_idx][frame][0] < z_min:
                    z_min = df[hand_idx][frame][2]
                if df[hand_idx][frame][0] > z_max:
                    z_max = df[hand_idx][frame][2]

    if len(df['R_Hand'][0]) > 2:
        # get range
        x_len = x_max - x_min
        y_len = y_max - y_min
        z_len = z_max - z_min
        # get volume
        vol = x_len * y_len * z_len
    else:
        x_len = x_max - x_min
        y_len = y_max - y_min
        # get area (ie volume)
        vol = x_len * y_len
    return vol



In [12]:
volume = calc_volume_size(df_OP,'B')
print('Volumetric space of the two hands: ' + str(volume) + " meters")
volume_R = calc_volume_size(df_OP,'R')
print('Volumetric space of the right hand: ' + str(volume_R) + " meters")

Volumetric space of the two hands: 2.252474567853439e-05 meters
Volumetric space of the right hand: 2.940706854339959e-05 meters


<h2> Working with Multiple Files </h2>

In [13]:
# !pip install pympi-ling

In [14]:
import pympi

In [15]:
# this is usually the name of the video, and should be the same as 
# the name of the motion tracking data and the annotation file
# filename_short = ""

# df_MT = pd.read_csv(os.path.join(filename_short + ".csv"))
# df_MT_conv = convert_MP_to_OP(df_MT)                                   

# first we need to access our annotations

path = 'p:/shared/FOCUS-GROUPS/Gesture-Kinematics/gesture_space_compuation/'

MT_files = [file for file in os.listdir(path) if file.endswith(".csv")]
eaf_files = [file for file in os.listdir(path) if file.endswith(".eaf")]

results_df = pd.DataFrame(columns = ["file", "gesture", 
                            
                            "MN_mode_L","MN_mode_R", "volume_B", "volume_R", "volume_L"])

for MT_file in MT_files: 
    print('Processing file: ' + MT_file)

    df_MT = pd.read_csv(path + MT_file)
    df_OP = convert_MP_to_OP(df_MT)

    eaf_file = [eaf_file for eaf_file in eaf_files if eaf_file.startswith(MT_file.split("_")[0])][0]           

    eafob = pympi.Elan.Eaf(path + eaf_file)

    # make sure you set the name of the tier where the annotations of interest are located
    gesture_annots = eafob.get_annotation_data_for_tier("Director_Speech")

    # then we can look through each annotation, and calculate our kinematic values
    file_list = []
    g_index_list = []
    PV_R_list = []

    mode_L_list = []
    mode_R_list = []

    volume_B_list = []
    volume_R_list = []
    volume_L_list = []

    g_index = 1
    for annot in gesture_annots:
        # this first line just takes the rows that correspond to our annotation
        g_data = df_OP[(df_OP.time >= annot[0]) & (df_OP.time <= annot[1])]
        g_data.reset_index(inplace=True)
        
        if len(g_data) > 10:
        
            
            space_use_L, space_use_R, mcneillian_maxL, mcneillian_maxR, mcneillian_modeL, mcneillian_modeR = calc_mcneillian_space(g_data, "B")

            volume_B = calc_volume_size(g_data,'B')
            volume_R = calc_volume_size(g_data,'R')
            volume_L = calc_volume_size(g_data,'L')

            # now store them all in a dataframe
            file_list.append(MT_file)
            g_index_list.append(g_index)
        
            mode_L_list.append(mcneillian_modeL)
            mode_R_list.append(mcneillian_modeR)

            volume_B_list.append(volume_B)
            volume_R_list.append(volume_R)
            volume_L_list.append(volume_L)

            g_index += 1
        
    results_df = pd.concat([results_df, pd.DataFrame(np.column_stack([file_list, g_index_list,
                                    
                                    mode_L_list,  mode_R_list, volume_B_list, volume_R_list, volume_L_list]), columns = ["file", "gesture", 
                            
                            "MN_mode_L","MN_mode_R", "volume_B", "volume_R", "volume_L"])])
                    
    

Processing file: 1130_JS_body.csv
Processing file: 1349_HS_body.csv
Processing file: 5014_RS_body.csv
Processing file: 5137_JJ_body.csv


In [16]:
results_df

,file,gesture,MN_mode_L,MN_mode_R,volume_B,volume_R,volume_L
0,1130_JS_body.csv,1,4,4,0.002182778882217155,0.00047446273066396595,2.5719277690959524e-05
1,1130_JS_body.csv,2,4,4,1.0424387842444919e-07,8.918322138164783e-08,3.266759161986881e-08
2,1130_JS_body.csv,3,4,2,-0.0059152082579570255,-0.0028460319169525297,-0.00241502597991591
3,1130_JS_body.csv,4,4,4,-0.0001595410132358914,-3.607588681524172e-05,-6.83862847140297e-05
4,1130_JS_body.csv,5,4,4,0.0008193921514435599,0.0003960360937682999,0.001335409045228477
5,1130_JS_body.csv,6,4,2,-6.670071064714372e-05,0.00025007481295318814,-1.3601966739350594e-05
6,1130_JS_body.csv,7,4,4,0.0005057323952802051,0.00010329742696205666,0.0002616140415504896
0,1349_HS_body.csv,1,4,4,-0.0001302969000135424,-7.4792379775238944e-06,-4.297776502863312e-05
1,1349_HS_body.csv,2,4,4,0.0011088831329204647,-0.0009956326776556996,0.0002824630413489878
2,1349_HS_body.csv,3,4,4,-2.4307440832297752e-05,-1.0387699060391375e-05,-6.179707104337348e-06


In [17]:
# !conda install anaconda::openpyxl

In [18]:
results_df.to_excel('space_results.xlsx')